# **Function**

In [469]:
# fungsi load data txt
def load_data_to_list_lower(path_doc):
  path_doc = open(path, "r")
  documents_list = []
  for line in path_doc.readlines():
    lower = line.lower()
    text = lower.strip()
    documents_list.append(text)
  return documents_list

In [470]:
# fungsi menghapus karakter tidak penting pada list
def hapus_karakter_tidak_penting(dokumen_list,karakter_dihapus):
  while(karakter_dihapus in dokumen_list) : 
    dokumen_list.remove(karakter_dihapus) 
  return dokumen_list 

In [471]:
# cari index kata tertentu didalam list
def cari_index_string(dokumen_list,cari_string1,cari_string2,cari_string3,cari_string4):
  index_kata = 0
  for line in dokumen_list:
    index_kata = index_kata + 1
    if cari_string1 in line: 
      break
    elif cari_string2 in line: 
      break
    elif cari_string3 in line: 
      break
    elif cari_string4 in line: 
      break
  return index_kata

def cari_index_string2(dokumen_list,cari_string1,cari_string2,cari_string3,cari_string4,cari_string5,cari_string6):
  index_kata = 0
  for line in dokumen_list:
    index_kata = index_kata + 1
    if cari_string1 in line: 
      break
    elif cari_string2 in line: 
      break
    elif cari_string3 in line: 
      break
    elif cari_string4 in line: 
      break
    elif cari_string5 in line: 
      break
    elif cari_string6 in line: 
      break
  return index_kata

In [472]:
# get bagian dokumen (cari batasan bagian dokumen, misal : gugatan, putusan, amar, dll)
def get_bagian_dokumen(dokumen_list,start,end):
  start = cari_index_string(dokumen_list,start[0],start[1],start[2],start[3]);
  end = cari_index_string(dokumen_list,end[0],end[1],end[2],end[3]);
  dokumen = dokumen_list[start:end]
  return dokumen

def get_bagian_dokumen2(dokumen_list,start,end):
  start = cari_index_string2(dokumen_list,start[0],start[1],start[2],start[3],start[4],start[5]);
  end = cari_index_string2(dokumen_list,end[0],end[1],end[2],end[3],end[4],end[5]);
  dokumen = dokumen_list[start:end-1]
  return dokumen

In [473]:
# untuk menghapus footer, halaman, dan disclaimer
def hapus_discalimer(dokumen_list):
  dokumen_list = dokumen_list
  count = 0
  for i in dokumen_list: 
    if i == "disclaimer":
      dokumen_list[count-1] = ""
      dokumen_list[count] = ""
      dokumen_list[count+1] = ""
      dokumen_list[count+2] = ""
      dokumen_list[count+3] = ""
      dokumen_list[count+4] = ""
    count = count+1
  hapus_karakter_tidak_penting(dokumen_list,"")
  return dokumen_list

In [474]:
def pre_processing_doc(path,kata_awal,kata_akhir):
  # load data
  dokumen_bersih = load_data_to_list_lower(path)

  # hapus karakter tidak penting
  hapus_karakter_tidak_penting(dokumen_bersih,"")

  # batasi bagian dokumen yang akan di summerisation dengan kata kunci awal dan akhir, 
  # disetting harus 4 kata kunci, bila ingin tambah atau dikurangi edit dibagian fungsi
  dokumen_putusan = get_bagian_dokumen(dokumen_bersih,kata_awal,kata_akhir)

  # menghapus footer, halaman, dan disclaimer
  dokumen_putusan = hapus_discalimer(dokumen_putusan)
  return dokumen_putusan

In [475]:
# cari nama pemohon kasasi
def cari_nama_kasasi(kasasi):
  nama_pemohon = kasasi.split(", ")
  nama_pemohon = nama_pemohon[0]
  nama_pemohon = nama_pemohon.split(" ")
  nama_pemohon = " ".join(nama_pemohon)
  return nama_pemohon

In [476]:
# pembobotan matriks
def pembobotan_matriks(dokumen_putusan_list):
  from sklearn.feature_extraction.text import CountVectorizer
  vectorizer = CountVectorizer()
  bag_of_words = vectorizer.fit_transform(dokumen_putusan_list)
  bag_of_words.todense()
  return bag_of_words

In [477]:
# dekomposisi matriks
def dekomposisi_matriks(hasil_pembobotan_matriks):
  from sklearn.decomposition import TruncatedSVD
  svd = TruncatedSVD(n_components = 1)
  decomposition = svd.fit_transform(hasil_pembobotan_matriks)
  return decomposition

In [478]:
# normalisasi data
def normalisasi_data_dan_input_ke_table(dokumen_putusan_clean,hasil_dekomposisi):
  import pandas as pd
  topic_encoded_df = pd.DataFrame(hasil_dekomposisi, columns=["bobot_dokumen"])
  topic_encoded_df["dokumen_clean"]= dokumen_putusan_clean
  topic_encoded_df["bobot_dokumen"] = topic_encoded_df["bobot_dokumen"] / topic_encoded_df["bobot_dokumen"].max()
  return topic_encoded_df

# normalisasi data
def normalisasi_data_dan_input_ke_table2(hasil_dekomposisi):
  import pandas as pd
  topic_encoded_df = pd.DataFrame(hasil_dekomposisi, columns=["bobot_dokumen"])
  topic_encoded_df["bobot_dokumen"] = topic_encoded_df["bobot_dokumen"] / topic_encoded_df["bobot_dokumen"].max()
  return topic_encoded_df

In [479]:
# join list
def join_list_2_dimensi(list_data):
  for i in range(len(list_data)):
    list_data[i] = "".join(list_data[i])
  return list_data

In [480]:
# batasi angka desimal di pandas
def batasi_desimal_pandas(data_frame_pandas,nama_colom):
  for i in range(0,len(data_frame_pandas)):
    bobot = "{:.4f}".format(data_frame_pandas.loc[i,nama_colom])
    data_frame_pandas.loc[i,nama_colom] = float(bobot)
  return data_frame_pandas

In [481]:
# hapus penomoran baik angka, nomor dari hasil rangkuman
def hapus_bullets_and_numbering(dokumen_terpilih_lsa):
  hasil_rangkum_lsa = dokumen_terpilih_lsa
  count = 0
  for i in hasil_rangkum_lsa:
    index = 0
    for text in i:
        if text == "a." or text == "b." or text == "c." or text == "d." or text == "e." or text == "f." or text == "g.":
          hasil_rangkum_lsa[count][index] = ""
        elif text == "h." or text == "i." or text == "j." or text == "k." or text == "l." or text == "m." or text == "n.":
          hasil_rangkum_lsa[count][index] = ""
        elif text == "o." or text == "p." or text == "q." or text == "r." or text == "s." or text == "t." or text == "u.":
          hasil_rangkum_lsa[count][index] = ""
        elif text == "v." or text == "w." or text == "x." or text == "y." or text == "z.":
          hasil_rangkum_lsa[count][index] = ""
        elif text == "1." or text == "2." or text == "3." or text == "4." or text == "5." or text == "6." or text == "7.":
          hasil_rangkum_lsa[count][index] = ""
        elif text == "8." or text == "9." or text == "10." or text == "11." or text == "12." or text == "13.":
          hasil_rangkum_lsa[count][index] = ""
        elif text == "14." or text == "15." or text == "16." or text == "17." or text == "18." or text == "19." or text == "20.":
          hasil_rangkum_lsa[count][index] = ""
        elif text == "21." or text == "22." or text == "23." or text == "24." or text == "25." or text == "26." or text == "27.":
          hasil_rangkum_lsa[count][index] = ""
        elif text == "- " or text == "-" or text == " - ":
          hasil_rangkum_lsa[count][index] = ""
        else:
          hasil_rangkum_lsa[count][index] = text
        index +=1
    count += 1
    for i in hasil_rangkum_lsa:
      hapus_karakter_tidak_penting(i,"")
    for i in range(len(hasil_rangkum_lsa)):
      hasil_rangkum_lsa[i] = " ".join(hasil_rangkum_lsa[i])
    return hasil_rangkum_lsa

In [482]:
# fungsi CLSA
def CLSA(dokumen_hasil_preprocessing):
  # pembobotan matriks
  hasil_pembobotan_matriks = pembobotan_matriks(dokumen_hasil_preprocessing)

  # dekomposisi matriks
  hasil_dekomposisi = dekomposisi_matriks(hasil_pembobotan_matriks)

  # encoder data
  endcoder_data = normalisasi_data_dan_input_ke_table(dokumen_hasil_preprocessing,hasil_dekomposisi)
  endcoder_data = batasi_desimal_pandas(endcoder_data,'bobot_dokumen')

  # ekstraksi hasil rangkuman_clsa
  mean = endcoder_data["bobot_dokumen"].mean()
  compression_rate = mean
  rangkuman_clsa=endcoder_data[endcoder_data["bobot_dokumen"]>=compression_rate]
  rangkuman_clsa[['dokumen_clean','bobot_dokumen']]

  hasil_clsa = []
  for i in rangkuman_clsa['dokumen_clean']:
    hasil_clsa.append(i.split())

  # join kalimat
  hasil_clsa = hapus_bullets_and_numbering(hasil_clsa)
  hasil_clsa = ". ".join(hasil_clsa)
  return hasil_clsa

In [483]:
# fungsi LSA
def LSA(dokumen_hasil_preprocessing,compression_rate):
  # pembobotan matriks
  hasil_pembobotan_matriks = pembobotan_matriks(dokumen_hasil_preprocessing)

  # dekomposisi matriks
  hasil_dekomposisi = dekomposisi_matriks(hasil_pembobotan_matriks)

  # encoder data
  endcoder_data = normalisasi_data_dan_input_ke_table(dokumen_hasil_preprocessing,hasil_dekomposisi)
  endcoder_data = batasi_desimal_pandas(endcoder_data,'bobot_dokumen')

  # ekstraksi hasil rangkuman_lsa
  compression_rate = compression_rate #40%
  rangkuman_lsa=endcoder_data[endcoder_data["bobot_dokumen"]>=compression_rate]
  rangkuman_lsa[['dokumen_clean','bobot_dokumen']]

  hasil_lsa = []
  for i in rangkuman_lsa['dokumen_clean']:
    hasil_lsa.append(i.split())

  # join kalimat
  hasil_lsa = hapus_bullets_and_numbering(hasil_lsa)
  hasil_lsa = ". ".join(hasil_lsa)
  return hasil_lsa

In [484]:
def min_max_avg_eval(narasi_pd):
  narasi_tranpose = narasi_pd
  mean_p = narasi_tranpose['Preccsion'].astype(float)
  mean_p = mean_p.mean()*100
  mean_r = narasi_tranpose['Recall'].astype(float)
  mean_r = mean_r.mean()*100
  mean_f = narasi_tranpose['Rouge_Score'].astype(float)
  mean_f = mean_f.mean()*100

  max_p = narasi_tranpose['Preccsion'].astype(float)
  max_p = max_p.max()*100
  max_r = narasi_tranpose['Recall'].astype(float)
  max_r = max_r.max()*100
  max_f = narasi_tranpose['Rouge_Score'].astype(float)
  max_f = max_f.max()*100

  min_p = narasi_tranpose['Preccsion'].astype(float)
  min_p = min_p.min()*100
  min_r = narasi_tranpose['Recall'].astype(float)
  min_r = min_r.min()*100
  min_f = narasi_tranpose['Rouge_Score'].astype(float)
  min_f = min_f.min()*100

  print("Rata rata  : ")
  print("ROUGE Precision  : {:.3f} %".format(mean_p))
  print("ROUGE Recall     : {:.3f} %".format(mean_r))
  print("\nNilai Max: ")
  print("ROUGE Precision  : {:.3f} %".format(max_p))
  print("ROUGE Recall     : {:.3f} %".format(max_r))
  print("\nNilai Min: ")
  print("ROUGE Precision  : {:.3f} %".format(min_p))
  print("ROUGE Recall     : {:.3f} %".format(min_r))

In [485]:
def eval_rouge_1(narasi_pd):
  narasi_tranpose = narasi_pd
  mean_f = narasi_tranpose['Rouge_Score'].astype(float)
  mean_f = mean_f.mean()*100
  max_f = narasi_tranpose['Rouge_Score'].astype(float)
  max_f = max_f.max()*100
  min_f = narasi_tranpose['Rouge_Score'].astype(float)
  min_f = min_f.min()*100

  print("Rouge-1 Score")
  print("Rata rata       : {:.3f} %".format(mean_f))
  print("Nilai Max       : {:.3f} %".format(max_f))
  print("Nilai Min       : {:.3f} %".format(min_f))

In [486]:
def uji_rouge_1(pakar_nomor,rangkuman_sistem_list,hasil_pakar_list):
  scor_rouge = {}
  for i in range(0,len(pakar_nomor)):
    # deklarasi rangkuman
    sistem = rangkuman_sistem_list[i]
    pakar = hasil_pakar_list[i]
    pakar = pakar.lower()

    # uji evaluasi rogue
    scorerr = rouge_scorer.RougeScorer(['rouge1'], use_stemmer=True)
    scorerr = scorerr.score(sistem,pakar)

    # simpan nilai ke dalam list
    a = scorerr['rouge1']
    p = "{:.3f}".format(a[0])
    r = "{:.3f}".format(a[1])
    f = "{:.3f}".format(a[2])

    # simpan nilai ke dalam dictionary
    scor = [sistem,pakar,p,r,f]

    # # pelabelan dokumen
    nama = "nomor_putusan_{}".format(pakar_nomor[i])
    scor_rouge[nama] = scor
  return scor_rouge

In [487]:
def plotting_evaluasi(dataframe_data,judul):
  pp = dataframe_data
  p = []
  for i in pp['Preccsion']:
    i = float(i)
    a = i*100
    p.append(a)

  r = []
  for j in pp['Recall']:
    j = float(j)
    b = j*100
    r.append(b) 

  data = {'number': range(0,50),
          'precission': p,
          'recall': r,
        }
    
  df = pd.DataFrame(data,columns=['number','precission','recall'])
  df.plot(x ='number', y=['precission','recall'] , kind = 'line', figsize=(20,10),fontsize=20)
  plt.xlabel('Dokumen', size=30)
  plt.ylabel('Nilai', size=30)
  plt.title("Hasil Evaluasi {}".format(judul),fontsize=20)
  plt.show()

In [488]:
def plotting_evaluasi_rouge(dataframe_data,judul):
  pp = dataframe_data
  r_score_lsa = []
  for i in pp['r_score_lsa']:
    r_score_lsa.append(float(i))

  r_score_clsa = []
  for j in pp['r_score_clsa']:
    r_score_clsa.append(float(j)) 

  data = {'number': range(0,50),
          'r_score_lsa': r_score_lsa,
          'r_score_clsa': r_score_clsa,
        }
    
  df = pd.DataFrame(data,columns=['number','r_score_lsa','r_score_clsa'])
  df.plot(x ='number', y=['r_score_lsa','r_score_clsa'] , kind = 'line', figsize=(20,10),fontsize=20)
  plt.xlabel('Dokumen', size=30)
  plt.ylabel('Nilai', size=30)
  plt.title("Hasil Evaluasi {}".format(judul),fontsize=20)
  plt.show()

In [489]:
# def perhitungan_rouge(pakar,sistem):
#   rang_sistem = sistem.split(" ")  
#   rang_pakar = pakar.split(" ") 
#   match = 0
#   for gram in rang_sistem:
#     if gram in rang_pakar:
#       match += 1
#   p = match/len(rang_pakar)
#   return p

# def rouge(rangkuman_sistem_list,rangkuman_pakar_list):
#   rouge_1 = []
#   panjang = len(rangkuman_sistem_list)
#   for i in range(panjang):
#     rouge_s = perhitungan_rouge(rangkuman_sistem_list[i],rangkuman_pakar_list[i])
#     rouge_1.append(rouge_s)

#   max_value = rouge_1[0]
#   for i in range(len(rouge_1)):
#     if i == len(rouge_1):
#       break
#     else:
#       if max_value > rouge_1[i]:
#         max_value = max_value
#       else:
#         max_value = rouge_1[i]
#   normalisasi_score = []
#   for i in rouge_1:
#     score = i/max_value
#     score = score*100
#     score = "{:.2f}".format(score)
#     score = float(score)
#     normalisasi_score.append(score)
#   return normalisasi_score

In [490]:
# def max_value(angka_list):
#   max_value = angka_list[0]
#   for i in range(len(angka_list)):
#     if i == len(angka_list):
#       break
#     else:
#       if max_value > angka_list[i]:
#         max_value = max_value
#       else:
#         max_value = angka_list[i]
#   return max_value
  
# def min_value(angka_list):
#   min_value = angka_list[0]
#   for i in range(len(angka_list)):
#     if i == len(angka_list):
#       break
#     else:
#       if min_value < angka_list[i]:
#         min_value = min_value
#       else:
#         min_value = angka_list[i]
#   return min_value